In [3]:
from keybert import KeyBERT
from transformers import pipeline
import pandas as pd
import seaborn as sns
import pandas as pd
import mysql.connector
import json
from sqlalchemy import create_engine

In [4]:
# Load database configuration from JSON file
with open('config.json') as config_file:
    config = json.load(config_file)['database']

# Establish a connection using the loaded configuration
cnx = mysql.connector.connect(**config)

# Define your query
query = f"SELECT * FROM gxs_apple_app_reviews"

# Use pandas to load sql query into a DataFrame
gxsapple = pd.read_sql(query, con=cnx)

# Don't forget to close the connection when done
cnx.close()

C:\Users\AngJi\AppData\Local\Temp\ipykernel_30796\3134596842.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataset = pd.read_sql(query, con=cnx)


In [ ]:
# Load database configuration from JSON file
with open('config.json') as config_file:
    config = json.load(config_file)['database']

# Establish a connection using the loaded configuration
cnx = mysql.connector.connect(**config)

# Define your query
query = f"SELECT * FROM gxs_playstore_app_reviews"

# Use pandas to load sql query into a DataFrame
gxsplaystore = pd.read_sql(query, con=cnx)

# Don't forget to close the connection when done
cnx.close()

In [ ]:
gxsapple['title_review'] = gxsapple['title'] + ' : ' + gxsapple['review']

gxsapple['thumbsUp']= 0

gxsapple['developerResponse'] = gxsapple['developerResponse'].apply(lambda x: x.get('body') if isinstance(x, dict) else x)
gxsapple['developerResponse'] = gxsapple['developerResponse'].str.replace(r"\{'id': \d+, 'body':", "", regex=True)
gxsapple['developerResponse']=gxsapple['developerResponse'].str.rstrip("}")
gxsapple['developerResponse'] = gxsapple['developerResponse'].str.replace('"', '', regex=False)
gxsapple['developerResponse'] = gxsapple['developerResponse'].str.replace("'", '', regex=False)

gxsapple_dropped = gxsapple.drop(columns=['title', 'review', 'userName', 'isEdited'])

gxsplaystore_dropped=gxsplaystore.drop(columns=['reviewId','userName','userImage','reviewCreatedVersion','repliedAt','appVersion'])
neworder=['replyContent', 'score', 'content', 'thumbsUpCount', 'at']
gxsplaystore_dropped=gxsplaystore_dropped[neworder]

gxsplaystore_dropped=gxsplaystore_dropped.rename(columns={
    'content': 'content',
    'thumbsUpCount': 'thumbsUpCount',
    'replyContent': 'replyContent',
    'score': 'score',
    'at':'date'
})

gxsapple_dropped_renamed = gxsapple_dropped.rename(columns={
    'title_review': 'content',
    'thumbsUp': 'thumbsUpCount',
    'developerResponse': 'replyContent',
    'rating': 'score',
    'date':'date'
})

dataset = pd.concat([gxsplaystore_dropped, gxsapple_dropped_renamed], axis=0, ignore_index=True)
if dataset['replyContent'].apply(lambda x: isinstance(x, dict)).any():
    dataset['replyContent'] = dataset['replyContent'].apply(lambda x: json.dumps(x) if isinstance(x, dict) else x)


In [2]:
#dataset = pd.read_csv('combined_reviews.csv', index_col=0)

In [5]:
dataset.head()

,replyContent,score,content,thumbsUpCount,date
0,Hi there! Thank you for your 5-star rating rev...,5,I like how the app is really light and fast co...,0,NaT
1,"Hi Jerry, this feature isn't ready at the mome...",1,Cannot add card to google pay. Also cannot add...,0,NaT
2,Thanks for your feedback on mobile wallets and...,4,Generally good. However please add either NFC ...,0,NaT
3,Hi Eric. We're sorry to hear this. We would li...,1,"With continuous failed to log in,contacted cso...",0,NaT
4,Hi Hafidz Melek! Thank you for your 5-star rat...,5,I don't get the negative reviews here. I insta...,0,NaT


In [4]:
sa_model = 'ajiayi/finetuning-sentiment-model-bank_reviews-otherbank'
sentiment_pipeline = pipeline("sentiment-analysis", model=sa_model)

In [5]:
ke_model = KeyBERT()

In [6]:
ke_model.extract_keywords(dataset.content[50])

[('rating', 0.5417),
 ('ratings', 0.4779),
 ('rate', 0.3851),
 ('app', 0.3224),
 ('unfair', 0.1467)]

In [7]:
sentiment = []
keywords = []

for index, row in dataset.iterrows():
    s = sentiment_pipeline(row['content'])[0]['label']
    sentiment.append(s)
    k = ke_model.extract_keywords(row['content'])
    #k = [i[0] for i in k]
    if k == []:
        k = ''
    else: k = k[0][0]
    keywords.append(k)

dataset['sentiment'] = sentiment
dataset['keywords'] = keywords


In [8]:
dataset.head()

,replyContent,score,content,thumbsUpCount,sentiment,keywords
0,"Hey Jiaxing! Hope you love the new ""Favourites...",5,"Gxs is simple and easy to use, with a saving a...",13,POSITIVE,"[gxs, funds, debit, investment, card]"
1,We apologize for the inconvenience you had to ...,3,It always ask for finger identification even t...,0,NEGATIVE,"[finger, identification, ask, given]"
2,"Hi Sean, we understand your concern and we wou...",1,What I've installed on my phone is none of you...,0,NEGATIVE,"[apps, app, whitelist, phone, using]"
3,"Hi Xiangqing, we'd love to look into this for ...",1,cant logging using finger print . previously s...,0,NEGATIVE,"[logging, finger, print, update, using]"
4,"Hi Matt, thanks for reaching out! If you are r...",1,For 1st time registration signup by singpass a...,0,NEGATIVE,"[singpass, signup, registration, denied, appli..."


In [9]:
#dataset.to_csv('sentiment_data.csv')
# Load database configuration from JSON file
with open('config.json') as config_file:
    config = json.load(config_file)['database']

# Construct the connection string using the loaded configuration
user = config['user']
password = config['password']
host = config['host']
dbname = config['database']
port = config['port']
conn_string = f'mysql+mysqlconnector://{user}:{password}@{host}:{port}/{dbname}'

# Create an engine
engine = create_engine(conn_string)

dataset.to_sql(name='sentiment_data', con=engine, if_exists='replace', index=False)